In [9]:
import re

class SExpressionParser:
    def __init__(self):
        self.var_counter = 1  # Đếm số biến trung gian (?X1, ?X2, ...)

    def get_new_var(self):
        """Tạo biến trung gian mới."""
        var_name = f"?X{self.var_counter}"
        self.var_counter += 1
        return var_name

    def parse_s_expr(self, s_expr):
        """Chuyển đổi S-Expression thành danh sách lồng nhau."""
        s_expr = re.sub(r'\(', ' ( ', s_expr)
        s_expr = re.sub(r'\)', ' ) ', s_expr)
        tokens = s_expr.split()
        return self.build_tree(tokens)

    def build_tree(self, tokens):
        """Chuyển đổi danh sách token thành cây lồng nhau."""
        if not tokens:
            return None
        token = tokens.pop(0)
        if token == "(":
            sub_expr = []
            while tokens[0] != ")":
                sub_expr.append(self.build_tree(tokens))
            tokens.pop(0)  # Bỏ dấu ")"
            return sub_expr
        elif token == ")":
            raise ValueError("Unexpected ')'")
        else:
            return token

    def process_join(self, expr, target_var):
        """
        Xử lý JOIN, tạo triple SPARQL.
        """
        triples = []
        if not isinstance(expr, list):
            return expr, triples

        if expr[0] == "AND":
            # Xử lý từng JOIN trong AND riêng lẻ
            for sub_expr in expr[1:]:
                _, sub_triples = self.process_join(sub_expr, target_var)
                triples.extend(sub_triples)
            return target_var, triples

        if expr[0] == "JOIN":
            right_expr = expr[2]
            right_triples = []
            if isinstance(right_expr, list) and right_expr[0] == "JOIN":
                right_var, right_triples = self.process_join(right_expr, self.get_new_var())
            else: 
                right_var = right_expr
            
            # Xử lý nhánh trái
            left_expr = expr[1]
            if isinstance(left_expr, list) and left_expr[0] == "R":
                rel = left_expr[1]
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([right, f"wdt:{rel}", left])
            else:
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([left, f"wdt:{left_expr}", right])

            # Thêm các triples từ nhánh phải trước khi thêm triple chính
            triples = right_triples + triples
            return target_var, triples

        return expr, triples

    def s_expr_to_sparql(self, s_expr):
        """Chuyển đổi từ S-Expression sang SPARQL."""
        parsed_expr = self.parse_s_expr(s_expr)
        target_var = "?answer"
        final_var, triples = self.process_join(parsed_expr, target_var)

        sparql_body = "\n  ".join([" ".join(t) + " ." for t in triples])
        sparql_query = f"""PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT DISTINCT {target_var} WHERE {{ 
  {sparql_body}
}}"""
        return sparql_query


In [10]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON

In [11]:
def execute_query_with_odbc(sparql_query):
    """
    Thực thi truy vấn SPARQL trên Wikidata endpoint và trả về kết quả.
    """
    ENDPOINT_URL = "https://query.wikidata.org/sparql"
    if not sparql_query:
        return []

    sparql = SPARQLWrapper(ENDPOINT_URL)
    sparql.setQuery(str(sparql_query))  # Ép kiểu về str cho chắc chắn
    sparql.setReturnFormat(JSON)

    try:
        response = sparql.query().convert()
        answers = [item["answer"]["value"] for item in response["results"]["bindings"] if "answer" in item]
        return answers
    except Exception as e:
        return []


# # Đọc dữ liệu từ file LC-QuAD2.0.json
# with open("Data/LC-QuAD2.0/label_map/LC-QuAD2.0_test.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# parser = SExpressionParser()
# results = []
# count_with_answers = 0
# data = data[:500]
# for entry in data:
#     s_expr = entry.get("s_expr", None)
#     if s_expr:
#         sparql_query = parser.s_expr_to_sparql(s_expr)
#         answers = execute_query_with_odbc(sparql_query)

#         # Cập nhật lại entry
#         entry["sparql_new"] = sparql_query
#         entry["answers_new"] = answers

#         # Đếm số truy vấn có kết quả
#         if answers:
#             count_with_answers += 1

#     results.append(entry)

# # Lưu kết quả vào file mới
# with open("LC-QuAD2.0_with_sparql.json", "w", encoding="utf-8") as f:
#     json.dump(results, f, ensure_ascii=False, indent=4)

# print(f"Tổng số truy vấn có kết quả: {count_with_answers}/{len(data)}")

In [17]:
parser = SExpressionParser()
s_expr = "["
sparql_query = parser.s_expr_to_sparql(s_expr)
print(sparql_query)


PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT DISTINCT ?answer WHERE { 
  
}
